In [2]:
# Import packages and setup gpu configuration.
# This code block shouldnt need to be adjusted!
import os
import sys
import json
import yaml
import numpy as np
import copy
import math
import time
import random
from tqdm import tqdm
import webdataset as wds
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import transforms
import utils
from mae_utils.flat_models import *

# tf32 data type is faster than standard float32
torch.backends.cuda.matmul.allow_tf32 = True
# following fixes a Conv3D CUDNN_NOT_SUPPORTED error
torch.backends.cudnn.benchmark = True

## MODEL TO LOAD ##
model_name = "HCPflat_large_gsrFalse_"
parquet_folder = "epoch99"

# outdir = os.path.abspath(f'checkpoints/{model_name}')
outdir = os.path.abspath(f'checkpoints/{model_name}')

print("outdir", outdir)
# Load previous config.yaml if available
if os.path.exists(f"{outdir}/config.yaml"):
    config = yaml.load(open(f"{outdir}/config.yaml", 'r'), Loader=yaml.FullLoader)
    print(f"Loaded config.yaml from ckpt folder {outdir}")
    # create global variables from the config
    print("\n__CONFIG__")
    for attribute_name in config.keys():
        print(f"{attribute_name} = {config[attribute_name]}")
        globals()[attribute_name] = config[f'{attribute_name}']
    print("\n")

world_size = os.getenv('WORLD_SIZE')
if world_size is None: 
    world_size = 1
else:
    world_size = int(world_size)
print(f"WORLD_SIZE={world_size}")

if utils.is_interactive():
    # Following allows you to change functions in models.py or utils.py and 
    # have this notebook automatically update with your revisions
    %load_ext autoreload
    %autoreload 2

batch_size = probe_batch_size
num_epochs = probe_num_epochs

data_type = torch.float32 # change depending on your mixed_precision
global_batch_size = batch_size * world_size

device = torch.device('cuda')

print("PID of this process =",os.getpid())

utils.seed_everything(seed)

outdir /weka/proj-fmri/ckadirt/fMRI-foundation-model/src/checkpoints/HCPflat_large_gsrFalse_
Loaded config.yaml from ckpt folder /weka/proj-fmri/ckadirt/fMRI-foundation-model/src/checkpoints/HCPflat_large_gsrFalse_

__CONFIG__
base_lr = 0.001
batch_size = 32
ckpt_interval = 5
ckpt_saving = True
cls_embed = True
contrastive_loss_weight = 1.0
datasets_to_include = HCP
decoder_embed_dim = 512
grad_accumulation_steps = 1
grad_clip = 1.0
gsr = False
hcp_flat_path = /weka/proj-medarc/shared/HCP-Flat
mask_ratio = 0.75
model_name = HCPflat_large_gsrFalse_
no_qkv_bias = False
norm_pix_loss = False
nsd_flat_path = /weka/proj-medarc/shared/NSD-Flat
num_epochs = 100
num_frames = 16
num_samples_per_epoch = 200000
num_workers = 10
patch_size = 16
pct_masks_to_decode = 1
plotting = True
pred_t_dim = 8
print_interval = 20
probe_base_lr = 0.0003
probe_batch_size = 8
probe_num_epochs = 30
probe_num_samples_per_epoch = 100000
resume_from_ckpt = True
seed = 42
sep_pos_embed = True
t_patch_size = 2
test_nu

# hcp_flat

In [3]:
from mae_utils.flat import load_hcp_flat_mask
from mae_utils.flat import create_hcp_flat
import mae_utils.visualize as vis

if utils.is_interactive(): # Use less samples per epoch for debugging
    probe_num_samples_per_epoch = 100000
    test_num_samples_per_epoch = 100000
    num_epochs = 10


# Load ckpt
if not os.path.exists(outdir) or not os.path.isdir(outdir):
    assert True, (f"\nCheckpoint folder {outdir} does not exist.\n")
else:
    checkpoint_files = [f for f in os.listdir(outdir) if f.endswith('.pth')]

    # Find the latest ckpt to load
    epoch_numbers = []
    for file in checkpoint_files:
        try:
            epoch_number = int(file.split('epoch')[-1].split('.')[0])
            epoch_numbers.append(epoch_number)
        except ValueError:
            continue
    latest_epoch = max(epoch_numbers)
    checkpoint_name = f"epoch{latest_epoch}.pth"
    
    ### Or provide the specific checkpoint you want to load
    # checkpoint_name = "epoch10.pth" #"epoch15.pth"

    # Load the checkpoint
    checkpoint_path = os.path.join(outdir, checkpoint_name)
    state = torch.load(checkpoint_path)

model = mae_vit_large_fmri(
    patch_size=16,
    decoder_embed_dim=decoder_embed_dim,
    t_patch_size=t_patch_size,
    pred_t_dim=pred_t_dim,
    decoder_depth=4,
    cls_embed=cls_embed,
    norm_pix_loss=norm_pix_loss,
    no_qkv_bias=no_qkv_bias,
    sep_pos_embed=sep_pos_embed,
    trunc_init=trunc_init,
    img_mask=state["model_state_dict"]['img_mask']
)

model.load_state_dict(state["model_state_dict"], strict=True) #model_state_dict
print(f"\nLoaded checkpoint {checkpoint_name} from {outdir}\n")

model.eval()
model.requires_grad_(False)
model.to(device)
pass

/tmp/ipykernel_2748908/405765492.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(checkpoint_path)


img_size (144, 320) patch_size (16, 16) frames 16 t_patch_size 2
model initialized

Loaded checkpoint epoch99.pth from /weka/proj-fmri/ckadirt/fMRI-foundation-model/src/checkpoints/HCPflat_large_gsrFalse_



In [4]:
import argparse
import json
import os
import pickle
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

target = "trial_type"
print(f"Target: {target}")

train_features = pd.read_parquet(f"{outdir}/{parquet_folder}/HCP/train.parquet")
test_features = pd.read_parquet(f"{outdir}/{parquet_folder}/HCP_/test.parquet")

print(f"train: {train_features.shape}, test: {test_features.shape}")
print(f"test: {test_features.shape}")

X_train = np.stack(train_features["feature"])
X_test = np.stack(test_features["feature"])
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"X_test: {X_test.shape}")


if target == "task":
    labels_train = train_features["task"].str.rstrip("1234").values
    labels_test = test_features["task"].str.rstrip("1234").values
elif target == "trial_type":
    labels_train = train_features["trial_type"].values
    labels_test = test_features["trial_type"].values

label_enc = LabelEncoder()
y_train = label_enc.fit_transform(labels_train)
y_test = label_enc.transform(labels_test)

print(f"classes ({len(label_enc.classes_)}): {label_enc.classes_}")
print(
    f"\ny_train: {y_train.shape} {y_train[:20]}\n"
    f"y_test: {y_test.shape} {y_test[:20]}"
)
del train_features, test_features

train_ind, val_ind = train_test_split(
    np.arange(len(X_train)), train_size=0.9, random_state=42
)
print(
    f"\ntrain_ind: {len(train_ind)} {train_ind[:10]}\n"
    f"val_ind: {len(val_ind)} {val_ind[:10]}"
)
X_train, X_val = X_train[train_ind], X_train[val_ind]
y_train, y_val = y_train[train_ind], y_train[val_ind]

print("Fitting PCA projection")
pca = PCA(n_components=384, whiten=True, svd_solver="randomized")
pca.fit(X_train)

X_train = pca.transform(X_train)
X_val = pca.transform(X_val)
X_test = pca.transform(X_test)

print("Fitting logistic regression")
clf = LogisticRegressionCV()
clf.fit(X_train, y_train)

train_acc = clf.score(X_train, y_train)
val_acc = clf.score(X_val, y_val)
test_acc = clf.score(X_test, y_test)

result = {
    "target": target,
    "train_acc": train_acc,
    "val_acc": val_acc,
    "test_acc": test_acc,
}

with open(f"{outdir}/{parquet_folder}/HCP/downstream.json", 'w') as out_json:
    json.dump(result, out_json)

print(f"Done:\n{json.dumps(result)}")

Target: trial_type


NameError: name 'train_df' is not defined